In [ ]:
import mysql.connector
from neo4j import GraphDatabase

# --- MySQL and Neo4j Config ---
MYSQL_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',  # update if needed
    'database': 'appdbproj'
}

NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "neo4jneo4j")
NEO4J_DB = "actorsMarried"

# --- Connectors ---
mysql_conn = mysql.connector.connect(**MYSQL_CONFIG)
cached_studios = []
cursor = mysql_conn.cursor()
cursor.execute("SELECT StudioID, StudioName FROM Studio ORDER BY StudioID")
cached_studios = cursor.fetchall()
cursor.close()
neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

# --- Main Menu Functions ---
def view_directors_and_films():
    print("
[1] View Directors & Films")
    name_input = input("Enter director name (or part): ").strip()
    if not name_input:
        print("Returning to main menu.")
        return
    
    query = '''
        SELECT d.DirectorName, f.FilmName, s.StudioName
        FROM Director d
        JOIN Film f ON d.DirectorID = f.FilmDirectorID
        JOIN Studio s ON f.FilmStudioID = s.StudioID
        WHERE d.DirectorName LIKE %s
        ORDER BY d.DirectorName, f.FilmName
    '''

    cursor = mysql_conn.cursor()
    cursor.execute(query, (f"%{name_input}%",))
    results = cursor.fetchall()

    if not results:
        print("No directors match search string.")
    else:
        current_director = None
        for director, film, studio in results:
            if director != current_director:
                print(f"
Director: {director}")
                current_director = director
            print(f"  - Film: {film} (Studio: {studio})")
    cursor.close()

def view_actors_by_month():
    import calendar
    print("
[2] View Actors by Month of Birth")
    
    month_map = {m.lower()[:3]: i for i, m in enumerate(calendar.month_name) if m}

    while True:
        user_input = input("Enter month (1–12 or 3-letter name): ").strip().lower()

        if user_input.isdigit():
            month = int(user_input)
            if 1 <= month <= 12:
                break
            else:
                print("Invalid month. Must be 1–12.")
        elif user_input in month_map:
            month = month_map[user_input]
            break
        else:
            print("Invalid month. Please enter a number (1–12) or 3-letter month abbreviation.")

    cursor = mysql_conn.cursor()
    query = """
        SELECT ActorName, ActorDOB, ActorGender
        FROM Actor
        WHERE MONTH(ActorDOB) = %s
        ORDER BY ActorDOB;
    """
    cursor.execute(query, (month,))
    results = cursor.fetchall()
    cursor.close()

    if not results:
        print("No actors found born in that month.")
    else:
        for name, dob, gender in results:
            print(f"Name: {name}, DOB: {dob.strftime('%Y-%m-%d')}, Gender: {gender}")

def add_new_actor():
    from datetime import datetime
    print("
[3] Add New Actor")

    try:
        actor_id = int(input("Enter Actor ID: ").strip())
        cursor = mysql_conn.cursor()
        cursor.execute("SELECT 1 FROM Actor WHERE ActorID = %s", (actor_id,))
        if cursor.fetchone():
            print("Actor ID already exists.")
            cursor.close()
            return

        name = input("Enter Actor Name: ").strip()
        dob_input = input("Enter DOB (YYYY-MM-DD): ").strip()
        dob = datetime.strptime(dob_input, "%Y-%m-%d")
        gender = input("Enter Gender (M/F): ").strip().upper()
        country_id = int(input("Enter Country ID: ").strip())

        cursor.execute("SELECT 1 FROM Country WHERE CountryID = %s", (country_id,))
        if not cursor.fetchone():
            print("Invalid Country ID entered.")
            cursor.close()
            return

        query = '''
            INSERT INTO Actor (ActorID, ActorName, ActorDOB, ActorGender, ActorCountryID)
            VALUES (%s, %s, %s, %s, %s);
        '''
        cursor.execute(query, (actor_id, name, dob, gender, country_id))
        mysql_conn.commit()
        cursor.close()
        print("Actor successfully added.")

    except ValueError:
        print("Invalid input. Please enter correct types.")
    except mysql.connector.Error as err:
        print(f"Database error: {err}")

def view_married_actors():
    print("
[4] View Married Actors")
    try:
        actor_id = int(input("Enter Actor ID: ").strip())
        with neo4j_driver.session(database=NEO4J_DB) as session:
            query = """
                MATCH (a:Actor {ActorID: $aid})-[r:MARRIED_TO]-(b:Actor)
                RETURN a.ActorID AS A1, b.ActorID AS A2
            """
            result = session.run(query, aid=actor_id).single()
            if not result:
                print("Actor not married")
                return
            
            ids = [result['A1'], result['A2']]
            cursor = mysql_conn.cursor()
            cursor.execute("SELECT ActorID, ActorName FROM Actor WHERE ActorID IN (%s, %s)", tuple(ids))
            rows = cursor.fetchall()
            cursor.close()
            
            for row in rows:
                print(f"ActorID: {row[0]}, Name: {row[1]}")
    except ValueError:
        print("Invalid input. Please enter a valid Actor ID.")
    except Exception as e:
        print(f"Error: {e}")

def add_actor_marriage():
    print("
[5] Add Actor Marriage")
    while True:
        try:
            aid1 = int(input("Enter first Actor ID: ").strip())
            aid2 = int(input("Enter second Actor ID: ").strip())

            if aid1 == aid2:
                print("Actor cannot marry him/herself")
                continue

            cursor = mysql_conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM Actor WHERE ActorID IN (%s, %s)", (aid1, aid2))
            count = cursor.fetchone()[0]
            cursor.close()

            if count < 2:
                print("Invalid Actor IDs. Please try again.")
                continue

            with neo4j_driver.session(database=NEO4J_DB) as session:
                check_query = """
                    MATCH (a:Actor)
                    WHERE a.ActorID IN [$aid1, $aid2]
                    OPTIONAL MATCH (a)-[:MARRIED_TO]-(:Actor)
                    RETURN a.ActorID AS id, COUNT((a)-[:MARRIED_TO]-()) AS married
                """
                results = session.run(check_query, aid1=aid1, aid2=aid2).data()
                married_flags = {r['id']: r['married'] for r in results}

                m1 = married_flags.get(aid1, 0)
                m2 = married_flags.get(aid2, 0)

                if m1 and m2:
                    print("Both actors already married")
                    continue
                elif m1 or m2:
                    print("One actor already married")
                    continue

                create_query = """
                    MERGE (a:Actor {ActorID: $aid1})
                    MERGE (b:Actor {ActorID: $aid2})
                    CREATE (a)-[:MARRIED_TO]->(b)
                """
                session.run(create_query, aid1=aid1, aid2=aid2)
                print(f"Actor {aid1} is now MARRIED_TO actor {aid2}")
                break
        except ValueError:
            print("Invalid input. Please enter numeric Actor IDs.")
        except Exception as e:
            print(f"Error: {e}")

def view_studios():
    print("
[6] View Studios")
    if not cached_studios:
        print("No studios loaded.")
        return
    for sid, name in cached_studios:
        print(f"Studio ID: {sid}, Name: {name}")
    for sid, name in cached_studios:
        print(f"Studio ID: {sid}, Name: {name}")

def view_actor_count_by_country():
    print("
[7] View Actor Count by Country")
    try:
        cursor = mysql_conn.cursor()
        cursor.execute("""
            SELECT CountryName, COUNT(*) AS TotalActors
            FROM Actor
            JOIN Country ON ActorCountryID = CountryID
            GROUP BY CountryName
            ORDER BY TotalActors DESC;
        """)
        for name, count in cursor.fetchall():
            print(f"{name}: {count} actor(s)")
        cursor.close()
    except Exception as e:
        print(f"Error: {e}")

def exit_app():
    print("
Exiting application.")
    mysql_conn.close()
    neo4j_driver.close()
    exit()

# --- Main Menu Loop ---
def main():
    while True:
        print("""
MoviesDB
------------
====== Main Menu ======
1. View Directors & Films
2. View Actors by Month of Birth
3. Add New Actor
4. View Married Actors
5. Add Actor Marriage
6. View Studios
7. View Actor Count by Country
x. Exit Application
=======================""")
        choice = input("Enter your choice: ").strip()
        if choice == '1':
            view_directors_and_films()
        elif choice == '2':
            view_actors_by_month()
        elif choice == '3':
            add_new_actor()
        elif choice == '4':
            view_married_actors()
        elif choice == '5':
            add_actor_marriage()
        elif choice == '6':
            view_studios()
        elif choice == '7':
            view_actor_count_by_country()
        elif choice.lower() == 'x':
            exit_app()
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()
# end